# Strategy

By Nicolas Harvie

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import datetime

***
## Universe Scraper 
Grab top 20 market cap cryptocurrencies (scraped from CoinMarketCap)

In [2]:
###=== UNIVERSE SCRAPER ===###

url = "https://coinmarketcap.com/"
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table")

rows = table.find_all("tr")

ticker_list = []

for i, row in enumerate(rows[1:101]): # exclude header row
    if i < 10:
        ticker = row.find("a", {"class": "cmc-link"}).find('p', {'class': 'coin-item-symbol'}).text
    else:
        ticker = row.find("a", {"class": "cmc-link"}).find("span", {"class": "crypto-symbol"}).text

    ticker_list.append(ticker)

In [3]:
###=== CLEANING AND SELECTION OF THE UNIVERSE ===###

no_trades = ['USD', 'DAI', 'WBTC', 'WETH']

for i in range(len(no_trades)):    
    ticker_list = [x for x in ticker_list if no_trades[i] not in x]

top_twenty_mktcap = ticker_list[:20]

In [4]:
top_twenty_mktcap

['BTC',
 'ETH',
 'BNB',
 'XRP',
 'DOGE',
 'ADA',
 'SOL',
 'TRX',
 'MATIC',
 'SHIB',
 'DOT',
 'LTC',
 'TON',
 'BCH',
 'AVAX',
 'LINK',
 'LEO',
 'XLM',
 'UNI',
 'XMR']

**Available By Exchanges:** </br>
</br>
GMX V2: ETH, BTC, LINK, ARB, UNI, SOL, DOGE, XRP, LTC </br>
PERP V2: BTC, ETH, DOGE, AAVE, APE, ATOM, AVAX, BNB, CRV, FLOW, FTM, LINK, MATIC, NEAR, OP, PERP, SAND, SOL 


***
## Price Scraper

In [5]:
# Returns for past 14 days
start_date = (datetime.date.today() - datetime.timedelta(14)).strftime('%d-%m-%Y')
end_date = (datetime.date.today()).strftime('%d-%m-%Y')

### With CMC Scraper

In [6]:
from cryptocmd import CmcScraper

# Inital DF with Bitcoin 
scraper = CmcScraper("BTC", start_date, end_date)
df = pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':'BTC'})

for crypto in top_twenty_mktcap[1:]:
    scraper = CmcScraper(str(crypto), start_date, end_date)
    df = pd.merge(df, pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':str(crypto)}), left_index=True, right_index=True)
price_df = df[::-1]


***
## Strategy itself

In [7]:
def output_strategy(n_ls, data, output_sort=False):

    # Cumulative Returns for the Past Week
    log_ret_df = np.log(1 + data.pct_change())
    cum_returns = log_ret_df.apply(lambda x: x.sum())

    if output_sort == True:
        print(cum_returns.sort_values(ascending=False))
    

    top_stocks = cum_returns.sort_values(ascending=False).index[:n_ls].values
    bottom_stocks = cum_returns.sort_values(ascending=True).index[:n_ls].values

    print(f'Cryptos to long this week: {top_stocks}')
    print(f'Cryptos to short this week: {bottom_stocks}')


In [8]:
output_strategy(3, price_df, output_sort=True)

SHIB     0.193032
TON      0.176033
SOL      0.046571
LEO      0.004782
BTC     -0.009048
XMR     -0.012282
ETH     -0.014858
TRX     -0.014861
LINK    -0.028372
BNB     -0.029379
MATIC   -0.031817
DOT     -0.041185
AVAX    -0.041385
DOGE    -0.043350
BCH     -0.061660
UNI     -0.063813
ADA     -0.064427
XRP     -0.108835
XLM     -0.119493
LTC     -0.133827
dtype: float64
Cryptos to long this week: ['SHIB' 'TON' 'SOL']
Cryptos to short this week: ['LTC' 'XLM' 'XRP']
